In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.max_columns', 300)

scaler = pickle.load(open('scaler.pickle', 'rb'))
final_model = pickle.load(open('model.pickle', 'rb'))
selected_columns = pd.read_csv('selected_rfe.csv')

In [2]:
selected_columns.drop('Unnamed: 0', axis=1, inplace=True)

In [40]:
select_col = list(selected_columns['0'].values)

## Step 1: Read in hold out data, scalers, and best model

In [4]:
# holdout = pd.read_csv('resources/movies_holdout_features.csv', index_col=0)

In [5]:
# final_scaler = read_pickle(filename)
# final_model = read_pickle(filename)

In [6]:
hold_out = pd.read_csv("kc_house_data_test_features.csv")

1. Drop 'Unnamed: 0'
2. Get hold_out df to the same #of columns as traindf


In [7]:
hold_out

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,4318,263000018,20140521T000000,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
4319,4319,6600060120,20150223T000000,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
4320,4320,1523300141,20140623T000000,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
4321,4321,291310100,20150116T000000,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [8]:
hold_out.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
#Replace 33 bedrooms with 3 as it was a typo
hold_out['bedrooms'] = hold_out['bedrooms'].replace({33:3})

In [10]:
hold_out[(hold_out['yr_renovated']==0) & (hold_out['yr_built']==1990)]

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
99,1088801350,20150210T000000,3,2.50,2320,9610,1.0,0,0,3,9,1730,590,1990,0,98011,47.7394,-122.204,2450,9608
162,2744000010,20140513T000000,3,2.50,1950,8251,2.0,0,0,3,7,1950,0,1990,0,98001,47.3430,-122.280,1540,8588
240,925069134,20140910T000000,4,2.00,3090,41147,1.0,0,0,3,7,3090,0,1990,0,98052,47.6748,-122.049,3300,34280
325,6664900330,20150223T000000,3,2.50,1990,7577,2.0,0,0,3,7,1990,0,1990,0,98023,47.2908,-122.351,1900,7152
354,9406500600,20150205T000000,2,1.50,1068,1452,2.0,0,0,3,7,1068,0,1990,0,98028,47.7530,-122.244,1078,1357
611,1796381120,20140728T000000,3,2.00,1090,7350,1.0,0,0,4,7,1090,0,1990,0,98042,47.3687,-122.085,1490,7741
621,5468750040,20150126T000000,4,4.00,2740,8250,2.0,0,0,4,9,2740,0,1990,0,98042,47.3735,-122.156,2290,8250
680,1939100610,20140623T000000,4,2.50,2300,7989,2.0,0,0,3,9,2300,0,1990,0,98074,47.6273,-122.034,2280,8835
734,1939130120,20140718T000000,4,2.50,3100,8529,2.0,0,0,3,9,3100,0,1990,0,98074,47.6252,-122.029,2710,8344
760,7701961220,20140626T000000,4,2.50,2990,16809,2.0,0,0,3,11,2990,0,1990,0,98077,47.7123,-122.073,3340,18752


In [11]:
#Create new yr_renovated column removing 0 values in yr_renovated column to an actual year(yr_built)
hold_out['yr_renovated2'] = np.where(hold_out['yr_renovated']==0, hold_out['yr_built'], hold_out['yr_renovated'])

In [12]:
#Change date column to datetime type and create two additional columns for month and year sold
hold_out['date'] = pd.to_datetime(hold_out['date']) #turn date column to date type
hold_out['month'] = pd.DatetimeIndex(hold_out.loc[:,'date']).month #Create month column
hold_out['year'] = pd.DatetimeIndex(hold_out.loc[:,'date']).year #create year column

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [13]:
# transformed_holdout = final_scaler(holdout)

In [14]:
hold_out['yrs_since_built'] = 2020 - hold_out['yr_built']

In [15]:
hold_out['yrs_since_reno'] = 2020 - hold_out['yr_renovated2']

In [16]:
hold_out['yrs_btwn_built_reno'] = hold_out['yr_renovated2'] - hold_out['yr_built']

In [17]:
hold_out['yrs_since_sold'] = 2020 - hold_out['year']

In [18]:
hold_out['property_age'] = hold_out['year'] - hold_out['yr_built']

In [19]:
hold_out['has_basement'] = np.where(hold_out['sqft_basement']==0, 0, 1)

In [20]:
hold_out['grade_exp'] = np.exp(hold_out['grade'])

In [21]:
dummy_zip = pd.get_dummies(hold_out['zipcode'], drop_first=False)
dummy_zip.reset_index(inplace=True)
dummy_zip = dummy_zip.add_prefix("{}#".format('zipcode'))
dummy_zip = dummy_zip[['zipcode#98004','zipcode#98102','zipcode#98109','zipcode#98112','zipcode#98039','zipcode#98040','zipcode#98119', 'zipcode#98105','zipcode#98006','zipcode#98005']]

In [22]:
hold_out.drop('zipcode', axis=1, inplace=True)
hold_out = hold_out.join(dummy_zip)

In [23]:
dummy_grade = pd.get_dummies(hold_out['grade'])
dummy_grade = dummy_grade.add_prefix("{}#".format('grade'))
hold_out = hold_out.join(dummy_grade)

In [24]:
# your code here
hold_out['sqft_living_lot']= hold_out['sqft_living']*hold_out['sqft_lot']

In [25]:
main_features = ['bedrooms', 'bathrooms', 'sqft_living','sqft_lot', 
                 'floors', 'waterfront', 'view', 'condition', 'grade','grade_exp',
                 'has_basement','property_age' ,'zipcode#98004', 
                 'zipcode#98102', 'zipcode#98109', 'zipcode#98112','zipcode#98039',
                 'zipcode#98040', 'zipcode#98119', 'zipcode#98105', 'zipcode#98006',
                 'zipcode#98005']

In [26]:
features = main_features

In [27]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(hold_out[features])
poly2_columns = poly_2.get_feature_names(features)
hold_out_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)
hold_out_poly2.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,grade_exp,has_basement,property_age,zipcode#98004,zipcode#98102,zipcode#98109,zipcode#98112,zipcode#98039,zipcode#98040,zipcode#98119,zipcode#98105,zipcode#98006,zipcode#98005,bedrooms^2,bedrooms bathrooms,bedrooms sqft_living,bedrooms sqft_lot,bedrooms floors,bedrooms waterfront,bedrooms view,bedrooms condition,bedrooms grade,bedrooms grade_exp,bedrooms has_basement,bedrooms property_age,bedrooms zipcode#98004,bedrooms zipcode#98102,bedrooms zipcode#98109,bedrooms zipcode#98112,bedrooms zipcode#98039,bedrooms zipcode#98040,bedrooms zipcode#98119,bedrooms zipcode#98105,bedrooms zipcode#98006,bedrooms zipcode#98005,bathrooms^2,bathrooms sqft_living,bathrooms sqft_lot,bathrooms floors,bathrooms waterfront,bathrooms view,bathrooms condition,bathrooms grade,bathrooms grade_exp,bathrooms has_basement,bathrooms property_age,bathrooms zipcode#98004,bathrooms zipcode#98102,bathrooms zipcode#98109,bathrooms zipcode#98112,bathrooms zipcode#98039,bathrooms zipcode#98040,bathrooms zipcode#98119,bathrooms zipcode#98105,bathrooms zipcode#98006,bathrooms zipcode#98005,sqft_living^2,sqft_living sqft_lot,sqft_living floors,sqft_living waterfront,sqft_living view,sqft_living condition,sqft_living grade,sqft_living grade_exp,sqft_living has_basement,sqft_living property_age,sqft_living zipcode#98004,sqft_living zipcode#98102,sqft_living zipcode#98109,sqft_living zipcode#98112,sqft_living zipcode#98039,sqft_living zipcode#98040,sqft_living zipcode#98119,sqft_living zipcode#98105,sqft_living zipcode#98006,sqft_living zipcode#98005,sqft_lot^2,sqft_lot floors,sqft_lot waterfront,sqft_lot view,sqft_lot condition,sqft_lot grade,sqft_lot grade_exp,sqft_lot has_basement,sqft_lot property_age,sqft_lot zipcode#98004,sqft_lot zipcode#98102,sqft_lot zipcode#98109,sqft_lot zipcode#98112,sqft_lot zipcode#98039,sqft_lot zipcode#98040,sqft_lot zipcode#98119,sqft_lot zipcode#98105,sqft_lot zipcode#98006,sqft_lot zipcode#98005,floors^2,floors waterfront,floors view,floors condition,floors grade,floors grade_exp,floors has_basement,floors property_age,floors zipcode#98004,floors zipcode#98102,floors zipcode#98109,floors zipcode#98112,floors zipcode#98039,floors zipcode#98040,floors zipcode#98119,floors zipcode#98105,floors zipcode#98006,floors zipcode#98005,waterfront^2,waterfront view,waterfront condition,waterfront grade,waterfront grade_exp,waterfront has_basement,waterfront property_age,waterfront zipcode#98004,waterfront zipcode#98102,waterfront zipcode#98109,waterfront zipcode#98112,waterfront zipcode#98039,waterfront zipcode#98040,waterfront zipcode#98119,waterfront zipcode#98105,waterfront zipcode#98006,waterfront zipcode#98005,view^2,view condition,view grade,view grade_exp,view has_basement,view property_age,view zipcode#98004,view zipcode#98102,view zipcode#98109,view zipcode#98112,view zipcode#98039,view zipcode#98040,view zipcode#98119,view zipcode#98105,view zipcode#98006,view zipcode#98005,condition^2,condition grade,condition grade_exp,condition has_basement,condition property_age,condition zipcode#98004,condition zipcode#98102,condition zipcode#98109,condition zipcode#98112,condition zipcode#98039,condition zipcode#98040,condition zipcode#98119,condition zipcode#98105,condition zipcode#98006,condition zipcode#98005,grade^2,grade grade_exp,grade has_basement,grade property_age,grade zipcode#98004,grade zipcode#98102,grade zipcode#98109,grade zipcode#98112,grade zipcode#98039,grade zipcode#98040,grade zipcode#98119,grade zipcode#98105,grade zipcode#98006,grade zipcode#98005,grade_exp^2,grade_exp has_basement,grade_exp property_age,grade_exp zipcode#98004,grade_exp zipcode#98102,grade_exp zipcode#98109,grade_exp zipcode#98112,grade_exp zipcode#98039,grade_exp zipcode#98040,grade_exp zipcode#98119,grade_exp zipcode#98105,grade_exp zipcode#98006,grade_exp zipcode#98005,has_basement^2,has_basement property_age,has_basement zipcode#98004,has_basement zipcode#98102,has_basemen

In [28]:
hold_out_poly2.shape

(4323, 275)

In [29]:
# from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression

# selector = SelectKBest(f_regression, k=200)

In [30]:
# selected_columns = hold_out_poly2.columns[selector.get_support()]
# removed_columns = hold_out_poly2.columns[~selector.get_support()]

## Step 3: Predict the holdout set

3. At this point hold_out should be fully cleaned.


In [41]:
select_col

['bedrooms',
 'bathrooms',
 'floors',
 'waterfront',
 'view',
 'grade',
 'has_basement',
 'zipcode#98004',
 'zipcode#98102',
 'zipcode#98112',
 'zipcode#98039',
 'zipcode#98040',
 'zipcode#98119',
 'zipcode#98105',
 'zipcode#98006',
 'bedrooms^2',
 'bedrooms bathrooms',
 'bedrooms floors',
 'bedrooms waterfront',
 'bedrooms view',
 'bedrooms condition',
 'bedrooms grade',
 'bedrooms has_basement',
 'bedrooms property_age',
 'bedrooms zipcode#98004',
 'bedrooms zipcode#98102',
 'bedrooms zipcode#98109',
 'bedrooms zipcode#98112',
 'bedrooms zipcode#98039',
 'bedrooms zipcode#98040',
 'bedrooms zipcode#98119',
 'bedrooms zipcode#98105',
 'bedrooms zipcode#98006',
 'bathrooms^2',
 'bathrooms floors',
 'bathrooms waterfront',
 'bathrooms view',
 'bathrooms condition',
 'bathrooms grade',
 'bathrooms has_basement',
 'bathrooms property_age',
 'bathrooms zipcode#98004',
 'bathrooms zipcode#98102',
 'bathrooms zipcode#98109',
 'bathrooms zipcode#98112',
 'bathrooms zipcode#98039',
 'bathrooms

In [42]:
scaled_holdout = scaler.transform(hold_out_poly2[select_col])

In [43]:
preds = final_model.predict(scaled_holdout)

In [44]:
pd.DataFrame(preds).to_csv('housing_preds_sandrawelbeck.csv')

In [ ]:
# final_answers = final_model.predict(transformed_holdout)

## Step 4: Export your predictions

In [ ]:
# final_answer.to_csv('housing_preds_your_name.csv')